Import used modules

In [0]:
import os
import numpy as np
from keras.callbacks import ModelCheckpoint
from  PIL import Image
from tensorflow import keras
from tensorflow.keras.layers import  Dense, Flatten

Download dataset+preparing the dataset(Train,Val,Test)


In [0]:
!wget "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip"
!unzip "/content/kagglecatsanddogs_3367a.zip"
!rm "/content/PetImages/Cat/Thumbs.db"
!rm "/content/PetImages/Dog/Thumbs.db"
address_cats="/content/PetImages/Cat"
address_dogs="/content/PetImages/Dog"


# #preparing cat images
cats=[]
tmp=os.listdir(address_cats)
for i in tmp[:1000]:
  try:
    img=Image.open(address_cats+"/"+i)
    img = img.resize((224, 224))
    img=np.array(img) / 255.
    if img.shape==(224,224,3):
      cats.append(img)
  except:
    print("ignored loading ",i)

#preparing dog images
dogs=[]
tmp=os.listdir(address_dogs)
for i in tmp[:1000]:
  try:
    img=Image.open(address_dogs+"/"+i)
    img = img.resize((224, 224))
    img=np.array(img) / 255.

    if img.shape==(224,224,3):
       dogs.append(img)
  except:
    print("ignored loading ",i)
###ratio for dividing data into Train,Val,Test
size=len(dogs)
train_interval=range(0,int(0.6*size))
val_interval=range(int(0.6*size)+1,int(0.6*size)+1+int(0.2*size))
test_interval=(int(0.8*size)+1,size)


####preparing Train Set
x_train=[]
y_train=[]
for i in train_interval:
  x_train.append(cats[i])
  y_train.append(0)
for i in train_interval:
  x_train.append(dogs[i])
  y_train.append(1)
x_train=np.array(x_train)
y_train=np.array(y_train)
#preparing Val Set
x_val=[]
y_val=[]
for i in val_interval:
  x_val.append(cats[i])
  y_val.append(0)
for i in val_interval:
  x_val.append(dogs[i])
  y_val.append(1)
x_val=np.array(x_val)
y_val=np.array(y_val)
# #preparing Test Set
# x_test=[]
# y_test=[]
# for i in test_interval:
#   x_test.append(cats[i])
#   y_test.append(0)
# for i in test_interval:
#   x_test.append(dogs[i])
#   y_test.append(1)
# x_test=np.array(x_test)
# y_test=np.array(y_test)


In [0]:
model=keras.applications.vgg19.VGG19(include_top=False, weights='imagenet',input_shape=(224,224,3))
for i in model.layers:
  i.trainable=False
x = Flatten()(model.layers[-1].output)
x = Dense(1024, activation="selu")(x)
x=Dense(1, activation="sigmoid")(x)
model=keras.Model(model.input,x)


In [0]:
checkpoint = ModelCheckpoint("w.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy'])
model.fit(x=x_train,y=y_train,epochs=50,validation_data=(x_val,y_val),batch_size=256,callbacks=[checkpoint])


In [0]:
cats=[]
tmp=os.listdir(address_cats)
for i in tmp[1000:2000]:
  try:
    img=Image.open(address_cats+"/"+i)
    img = img.resize((224, 224))
    img=np.array(img) / 255.
    if img.shape==(224,224,3):
      cats.append(img)
  except:
    print("ignored loading ",i)

#preparing dog images
dogs=[]
tmp=os.listdir(address_dogs)
for i in tmp[1000:2000]:
  try:
    img=Image.open(address_dogs+"/"+i)
    img = img.resize((224, 224))
    img=np.array(img) / 255.

    if img.shape==(224,224,3):
       dogs.append(img)
  except:
    print("ignored loading ",i)

In [0]:

x_test=[]
y_test=[]
for i in range(len(cats)):
  x_test.append(cats[i])
  y_test.append(0)
for i in range(len(dogs)):
  x_test.append(dogs[i])
  y_test.append(1)
x_test=np.array(x_test)
y_test=np.array(y_test)
del cats
del dogs


In [0]:
model.evaluate(x_test,y_test)

1992/1992 [==============================] - 7s 3ms/sample - loss: 0.2689 - acc: 0.8916


[0.2688913242704418, 0.8915663]